# **CombinedTM 실습**

이번 노트북에서는 Top2Vec 모델을 한국어에 맞춰 사용할 수 있도록 Customizing한 후 인퍼런스 할 예정입니다. 인퍼런스 능력을 향상시키기 위해, 직접 BERT모델을 고르고, Tokenizer을 수정합니다. 이후, 결과를 직접 시계열에 맞춰 시각화한 후 토픽 모델링의 결과를 확인합니다.

## **1 - 준비작업**

### **1.1 - 라이브러리 설치**
필요한 라이브러리를 설치합니다. 별도로 수정하지 말아주세요.

In [ ]:
!pip install \
  contextualized_topic_models

!pip install \
  konlpy

!pip install \
  plotly

### **1.2 - 라이브러리 불러오기**
필요한 라이브러리를 불러옵니다. 별도로 수정하지 말아주세요.

In [83]:
import os
import csv
import numpy as np
import random
import torch
import pandas as pd
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing

from transformers import AutoModel, AutoTokenizer

from konlpy.tag import Kkma, Komoran

from typing import List, Tuple, Dict, Literal, Optional, Callable, Union

In [80]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
pd.set_option('mode.chained_assignment',  None) 

### **1.3 - Seed 고정**
결과를 일정하게 유지하기 위해 모든 SEED를 고정합니다. 별도로 수정하지 말아주세요.

In [6]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

### **1.4 - Colab**
코랩에 접근하기 위해 실행합니다. 그 전에, Colab 폴더에 코드와 데이터셋을 올바르게 넣었는지 확인해주세요.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## **2 - 데이터셋**

### **2.1 - 데이터셋 로드**
데이터셋을 드라이브에서 불러옵니다.

In [ ]:
PATH = './datasets/원자력+원전+탈원전_news_df.xlsx'
df1 = pd.read_excel(PATH)

PATH = './datasets/사설_반대.xlsx'
df2 = pd.read_excel(PATH)

PATH = './datasets/고리+신한울+새울+월성+신월성_news_df.xlsx'
df3 = pd.read_excel(PATH)

# df = pd.concat([df1, df2, df3], axis=0)

df = df2

미리보기

In [9]:
df.head()

,news_title,news_date,news_company,news_body
0,"[사설] `탈원전`, 방향은 맞지만 부작용 최소화해야",2017-05-29,디지털타임스,29일 국정기획자문위원회 이개호 경제2분과 위원장이 원자력안전위원회(원안위)에 원전...
1,[사설] 탈원전·석탄 조급증…안전하면서 값싼 전기는 없다,2017-05-30,한국경제,"문재인 정부의 에너지정책이 탈(脫)원전, 탈화력발전으로 급선회하고 있다. 그제 국정..."
2,[fn사설] 새 정부 탈원전 정책 우려 새겨 들어야,2017-06-01,파이낸셜뉴스,전문가 200명 비판 성명 방향 맞지만 너무 서둘러 에너지 분야 전문가들이 ...
3,[사설] 탈원전…정규직화…'밀어붙이기 정책'에서 놓치는 것들,2017-06-01,한국경제,"교수 230명 성명 ""에너지정책 전문가 논의 절실""일자리 해법, 누구나 비판하고 토..."
4,<사설>성급한 脫원전은 電力안보 해친다는 전문가들의 우려,2017-06-02,문화일보,문재인 정부의 탈(脫)원전 정책에 대해 에너지 전문가들이 정면으로 비판하고 나섰다....


### **2.2 - 텍스트 전처리**
해당 데이터는 크롤링된 데이터로, 올바른 결과를 위해서는 전처리가 필요합니다.
뉴스 기사를 위한 전처리 코드는 https://colab.research.google.com/drive/1O42v6YzHbTo_l36VG1xGp0aEuA0J8M3V?usp=sharing#scrollTo=P7JBkNna1jze 에서 가져왔습니다.

In [10]:
df['news_title'] = df['news_title'].map(lambda x: str(x))

In [11]:
import re, unicodedata
from string import whitespace
pattern_whitespace = re.compile(f'[{whitespace}]+')

df['news_title'] = df['news_title'].str.replace(pattern_whitespace, ' ').map(lambda x: unicodedata.normalize('NFC', x)).str.strip()


In [12]:
def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_others.sub('.', result)
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()

    return result

df['news_title'] = df['news_title'].map(clean_byline)

In [13]:
from string import whitespace, punctuation

def text_filter(text):
    punct_except_percent = ''.join([chr for chr in punctuation if chr != '%'])
    whitespace_convert_pattern = re.compile(f'[{whitespace}{punct_except_percent}]+')
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣]+')
    result = whitespace_convert_pattern.sub(' ', text)
    result = exclude_pattern.sub(' ', result).strip()
    result = whitespace_convert_pattern.sub(' ', result)
    return result

df['news_title'] = df['news_title'].map(text_filter)

전처리 후 결과 미리보기

In [14]:
df.head()

,news_title,news_date,news_company,news_body
0,탈원전 방향은 맞지만 부작용 최소화해야,2017-05-29,디지털타임스,29일 국정기획자문위원회 이개호 경제2분과 위원장이 원자력안전위원회(원안위)에 원전...
1,탈원전 석탄 조급증 안전하면서 값싼 전기는 없다,2017-05-30,한국경제,"문재인 정부의 에너지정책이 탈(脫)원전, 탈화력발전으로 급선회하고 있다. 그제 국정..."
2,새 정부 탈원전 정책 우려 새겨 들어야,2017-06-01,파이낸셜뉴스,전문가 200명 비판 성명 방향 맞지만 너무 서둘러 에너지 분야 전문가들이 ...
3,탈원전 정규직화 밀어붙이기 정책 에서 놓치는 것들,2017-06-01,한국경제,"교수 230명 성명 ""에너지정책 전문가 논의 절실""일자리 해법, 누구나 비판하고 토..."
4,성급한 원전은 안보 해친다는 전문가들의 우려,2017-06-02,문화일보,문재인 정부의 탈(脫)원전 정책에 대해 에너지 전문가들이 정면으로 비판하고 나섰다....


## **3 - 모델**

### **3.1 - BoW Embedding 만들기**

불용어 사전, Tokenizer을 생성하고, 이를 이용해 CountVectorizer을 만듭니다.

In [15]:
STOPWORDS_PATH = './datasets/user_stopwords.csv'

with open(STOPWORDS_PATH, 'r', encoding='cp949') as f:
    tmp = csv.reader(f)
    stop_words = set([l[0] for l in tmp])

In [16]:
USER_DICT_PATH = './datasets/user_dic.txt'

custom_dic = [
    '탈원전\tNNG',
]

with open(USER_DICT_PATH, 'w', encoding='utf-8') as f:
    for line in custom_dic:
        f.write(line + '\n')

In [17]:
class CustomTokenizer:

    def __init__(
        self,
        tagger: Callable = Komoran(), # KoNLPy의 태거가 들어갑니다.
        stop_words: Optional[List[str]] = None, # 불용어 사전이 들어갑니다.
        tag_list: List[str] = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'MM', 'MAG']
        ):
      self.tagger = tagger
      self.stop_words = stop_words
      self.tag_list = tag_list

    def __call__(self, text):
      text = text[:10000]
      # 명사, 형용사, 동사, 관형사와 일반 부사만 취한다.
      word_tokens = list(map(lambda x: x[0], filter(lambda x: x[1] in self.tag_list, self.tagger.pos(text))))
      result = [word for word in word_tokens if (word not in stop_words) and (len(word) >= 2)]
      return result

In [18]:
tagger = Komoran(userdic=USER_DICT_PATH)
tokenizer = CustomTokenizer(tagger = tagger, stop_words = stop_words)
vectorizer_model = CountVectorizer(tokenizer=tokenizer, max_features=2000)

vocab 사이즈를 2000으로 제한하고, vectorizer을 통해 각 문장마다 BoW Embedding을 만들어냅니다.

In [19]:
bow_embeddings = vectorizer_model.fit_transform(df['news_title'])

c:\Users\owj04\anaconda3\envs\AIKU\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


학습된 Vectorizer에서 vocab을 추출합니다.

In [21]:
vocab = vectorizer_model.get_feature_names_out() 
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [22]:
vocab

array(['10년', '10대', '13년', ..., '힘겹', '힘들', '힘쓰'], dtype=object)

### **3.2 - Contextualized Embedding 만들기**

기본으로 제공하는 SBERT는 영어 문장을 대상으로만 문장을 분류, 군집화 할 수 있습니다. <br>
한국어를 활용해 SBERT를 수행하고 싶다면, 한국어를 지원하는 BERT모델을 사용해야합니다. <br>
한국어를 지원하는 SBERT모델은 [여기]()에서 찾을수 있습니다.<br>
(이때, 주의할 점은 문장간의 유사도를 판별할 수 있는 모델을 사용해야 하기 때문에, 꼭 일반적인 BERT가 아닌 SBERT를 사용해야 합니다!)

**HuggingFace에서 한국어 SBERT모델 찾는법** <br>
- Hugging Face > Models > Natural Language Processing(좌측 메뉴)에 들어간다
- Filter by name에 "ko"라고 검색
- 검색된 다양한 모델에 들어간 후 원하는 모델을 정합니다.

**모델을 고르는 방법** <br>
- 모델의 성능 (주관적이지만 기능을 수행하기에 무리가 없는 수준이어야 합니다.)
- 어떤 데이터셋으로 학습되었는가? (내가 사용할 Domain과 비슷한 Corpus로 학습된 모델이 좋습니다.)

실습은 Hugging Face의 "jhgan/ko-sroberta-multitask"을 사용해서 진행하겠습니다.

In [23]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [24]:
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sroberta-multitask')

contextualized_embeddings = []
with torch.no_grad():
    for i in tqdm(df['news_title']):
        encoded_input = tokenizer(i, padding=True, truncation=True, return_tensors='pt')
        model_output = model(**encoded_input)
        contextualized_embeddings.append(mean_pooling(model_output, encoded_input['attention_mask']))

contextualized_embeddings = torch.cat(contextualized_embeddings)

100%|██████████| 2109/2109 [01:26<00:00, 24.37it/s]


### **3.3 - CTM 데이터셋 만들기**

In [25]:
qt = TopicModelDataPreparation()
training_dataset = qt.load(
    contextualized_embeddings=contextualized_embeddings, 
    bow_embeddings=bow_embeddings, 
    id2token=id2token
    )

모든 설정을 마쳤습니다. 이제 CTM모델을 생성해보도록 하겠습니다.

In [26]:
ctm = CombinedTM(
    bow_size=len(vocab), 
    contextual_size=768, 
    n_components=20, 
    num_epochs=100,
    batch_size=64,
    num_data_loader_workers=0
    )

### **3.4 - 모델 학습하기**

다른 모델과 다르게 CTM은 모델을 학습해야 합니다. GPU환경을 권장하며, RTX2060 환경에서 10000개의 데이터 기준 1분의 시간이 소요됩니다.

In [27]:
ctm.fit(training_dataset)

Epoch: [100/100]	 Seen Samples: [204800/210900]	Train Loss: 43.863324880599976	Time: 0:00:00.412094: : 100it [00:41,  2.42it/s]
100%|██████████| 33/33 [00:00<00:00, 104.08it/s]


## **4. 토픽 추출하기**

문서별로 어떤 토픽을 가지는지 알아보겠습니다.

`.get_topics()`로 토픽의 번호-토픽 키워드로 이루어진 딕셔너리를 얻을 수 있습니다.

In [29]:
idx2topic = ctm.get_topics()

df['topic_idx'] = np.argmax(ctm.get_doc_topic_distribution(training_dataset, n_samples=20), axis=1)
df['topic'] = df['topic_idx'].map(idx2topic)

100%|██████████| 33/33 [00:00<00:00, 96.75it/s]


In [30]:
df

,news_title,news_date,news_company,news_body,topic_idx,topic
0,탈원전 방향은 맞지만 부작용 최소화해야,2017-05-29,디지털타임스,29일 국정기획자문위원회 이개호 경제2분과 위원장이 원자력안전위원회(원안위)에 원전...,0,"[대만, 폐기, 독일, 투표, 탈원전, 포기, 실패, 교훈, 송영길, 교사]"
1,탈원전 석탄 조급증 안전하면서 값싼 전기는 없다,2017-05-30,한국경제,"문재인 정부의 에너지정책이 탈(脫)원전, 탈화력발전으로 급선회하고 있다. 그제 국정...",12,"[전력, 전기, 수급, 폭염, 인상, 비용, 급등, 탈원전, 시작, 예측]"
2,새 정부 탈원전 정책 우려 새겨 들어야,2017-06-01,파이낸셜뉴스,전문가 200명 비판 성명 방향 맞지만 너무 서둘러 에너지 분야 전문가들이 ...,11,"[탈원전, 에너지, 재생, 정책, 원자력, 폐기, 고집, 우려, 한국, 재검토]"
3,탈원전 정규직화 밀어붙이기 정책 에서 놓치는 것들,2017-06-01,한국경제,"교수 230명 성명 ""에너지정책 전문가 논의 절실""일자리 해법, 누구나 비판하고 토...",3,"[탈원전, 수원, 반대, 정책, 사장, 과학, 보복, 밀어붙이, 지적, 소주]"
4,성급한 원전은 안보 해친다는 전문가들의 우려,2017-06-02,문화일보,문재인 정부의 탈(脫)원전 정책에 대해 에너지 전문가들이 정면으로 비판하고 나섰다....,1,"[원전, 수원, 안전, 중단, 정비, 친환경, 일본, 폐기장, 이사회, 가동]"
...,...,...,...,...,...,...
2104,6개월 정부 초심으로 돌아가 국정 쇄신하라,2022-11-09,서울신문,첫 관문 ‘공정·상식’ 따른 참사 대처안보·경제 복합위기 극복 믿음 줘야\n\n\n...,6,"[대통령, 국정, 여야, 회동, 소통, 민심, 지지, 경제, 마지막, 임기]"
2105,힘겨웠던 정부 6개월 개혁의 신발끈 다시 매야,2022-11-10,한국경제,윤석열 정부가 출범 6개월을 맞았다. 국내외에서 안보와 경제가 뒤얽힌 복합적 위기 ...,4,"[정부, 정책, 실패, 국민, 바꾸, 책임, 전환, 떠넘기, 경제, 계기]"
2106,낙하산 논란에 사의 표명 한수원 사외이사 파행,2022-11-11,국제신문,고리 핵폐기장화 결정 초미의 관심…정권 바뀔 때마다 반복 이젠 달라야한국수력원자력(...,2,"[월성, 수사, 조작, 폐쇄, 밝히, 산업부, 블랙리스트, 의혹, 감사, 경제]"
2107,에너지 효율성 최악 산업 구조 전면 리셋해야,2022-11-12,서울경제,[서울경제] 국제 에너지 가격 폭등이 우리 경제를 위기로 몰아넣고 있다. 무역수지는...,16,"[공기업, 공공, 기업, 기관, 개혁, 경영, 부채, 일자리, 잔치, 재정]"


## **5 - 시각화**
LDA에서 사용하던 PyLDAvis를 사용합니다.

### **5.1 - 일반 시각화**

In [163]:
import pyLDAvis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10) 
ctm_pd = pyLDAvis.prepare(**lda_vis_data) 
pyLDAvis.display(ctm_pd)

100%|██████████| 33/33 [00:00<00:00, 93.47it/s]


### **5.2 - 시계열 시각화**

CTM은 기본으로 시계열 시각화를 지원하지 않으므로, 직접 함수를 만들어 진행하도록 하겠습니다.

In [186]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly as py
import plotly.offline as pyo
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
pio.renderers.default = 'notebook'
from datetime import datetime

In [206]:
def topics_over_time(
        topics: Union[pd.Series, List[int]],
        timestamps: Union[pd.Series, List[datetime]],
        idx2topic: Dict[int, str],
        freq: Literal['Y', 'M'] = 'M'
        ):
    timestamps = pd.cut(timestamps, bins=pd.date_range(timestamps.min(), timestamps.max(), freq=freq))
    df = pd.get_dummies(topics).groupby(timestamps).sum().reset_index()

    fig = go.Figure()
    for topic_idx in df.columns:
        if isinstance(topic_idx, str):
            continue

        if idx2topic:
            name = "_".join(idx2topic[topic_idx][:4])
        else:
            name = topic_idx

        fig.add_trace(go.Scatter(x=df.index, 
                                 y=df[topic_idx], 
                                 mode='lines+markers',
                                 fill = 'tozeroy',
                                 name=name))
    fig.update_layout(title='Topic',
                      xaxis_title='Year-Month',
                      yaxis_title='Frequency')
    
    fig.update_xaxes(range=[min(df.index), max(df.index)], tickvals=df.index, ticktext=[year_month.left.strftime('%Y-%m') for year_month in timestamps.cat.categories])
    fig.show()

In [207]:
topics = df['topic_idx']
timestamps = pd.to_datetime(df['news_date'])
topics_over_time(topics, timestamps, idx2topic=idx2topic)

## **수고하셨습니다**

더 많은 사용법은 [공식 GitHub](https://maartengr.github.io/BERTopic/index.html#installation)에서 찾아보세요!